In [ ]:
import requests
import json
import time

# for pokemon in dict_response
all_url = "https://pokeapi.co/api/v2/pokemon-species/"
wanted_games = {"red", "blue", "leafgreen", "white"}
pokemon_by_version = {version: [] for version in wanted_games}

# limit the calls, use pagination
offset = 0
limit = 100
session = requests.Session()

pokemon_urls = set()
while True:
    try:
        dict_response = requests.get(f"{all_url}?offset={offset}&limit={limit}").json()
    except requests.exceptions.ConnectionError:
        time.sleep(5)
        continue

    get_pokemon = dict_response["results"]

    # save the

    if len(get_pokemon) == 0:
        break

    pokemon_dict = {}

    for pokemon in get_pokemon:
        individual_pokemon = pokemon["url"]

        try:
            get_url = requests.get(individual_pokemon).json()
        except requests.exceptions.ConnectionError:
            time.sleep(5)
            continue

        if len(get_url["flavor_text_entries"]) > 0:
            for flavor_text_entry in get_url["flavor_text_entries"]:
                version_name = flavor_text_entry["version"]["name"]

                if version_name in wanted_games:
                    url = pokemon["url"]
                    pokemon_urls.add(url)

    if not dict_response["next"]:
        break
    else:
        offset += limit

print(pokemon_urls)


In [167]:
import requests
import time
import requests_cache
import re

all_url = "https://pokeapi.co/api/v2/pokemon-species/"
wanted_games = ["red", "blue" , "leafgreen", "white"]
pokemon_by_version = {version: [] for version in wanted_games}

offset = 0
#limit = 100 # 56 seconds
limit = 150

session = requests.Session()
requests_cache.install_cache() #56 seconds
pokemon_urls = set()

while True:
    try:
        dict_response = session.get(f"{all_url}?offset={offset}&limit={limit}").json()
    except requests.exceptions.ConnectionError:
        time.sleep(2**offset)
        continue

    get_pokemon = dict_response["results"]

    # save the

    if len(get_pokemon) == 0:
        break

    pokemon_dict = {}

    for pokemon in get_pokemon:
        individual_pokemon = pokemon["url"]

        try:
            get_url = session.get(individual_pokemon).json()
        except requests.exceptions.ConnectionError:
            time.sleep(2**offset)
            continue

        if len(get_url["flavor_text_entries"]) > 0:
            for flavor_text_entry in get_url["flavor_text_entries"]:
                version_name = flavor_text_entry["version"]["name"]

                if version_name in wanted_games:
                    url = pokemon["url"].split("/")[-2]
                    #checker = re.search(r'/(\d+)/$', url)
                    pokemon_urls.add(url)

    if not dict_response["next"]:
        break
    else:
        offset += limit

print(pokemon_urls)


KeyboardInterrupt: 

In [ ]:
print(len(pokemon_urls))

In [ ]:
import re

sample = 'https://pokeapi.co/api/v2/pokemon-species/1/'

checker = re.search(r'/(\d+)/$', url)
if checker:
    number = checker.group(1)
    print(number)  # output: 122




In [ ]:
for version in wanted_games:
    filename = f"pokemon_urls.json"
    data = list(pokemon_urls)
    data.sort()
    with open(filename, "w") as file:
        json.dump(data, file)


In [ ]:
import json

with open("pokemon_urls.json", "r") as file:
    data = json.load(file)

for url in data:
    print(url)

In [ ]:
import requests
import time
import requests_cache

pokemon_url = "https://pokeapi.co/api/v2/pokemon"
pokemon_list = []

session = requests.Session()
requests_cache.install_cache()


for pokemon in pokemon_urls:
    while True:
        try:
            pokemon_data = session.get(f"{pokemon_url}/{pokemon}").json()
            break
        except requests.exceptions.RequestException:
            time.sleep(2**offset)
            continue

    wanted_slots = [1, 2]  # list
    available_slots = [type_dict["slot"] for type_dict in pokemon_data["types"]]
    

    check = all(item in available_slots for item in wanted_slots)

    slots = {
        type_dict.get("slot"): type_dict.get("type").get("name")
        for type_dict in pokemon_data["types"]
        if type_dict.get("slot") in wanted_slots
    }

    stats = {
        "name": pokemon_data["name"].capitalize(),
        "id": pokemon_data["id"],
        "base_experience": pokemon_data["base_experience"],
        "weight": pokemon_data["weight"],
        "height": pokemon_data["height"],
        "order": pokemon_data["order"],
        "slot 1": slots.get(1),
        "bmi": round(pokemon_data["weight"] / (pokemon_data["height"] ** 2), 4),
        "front_default": pokemon_data["sprites"]["front_default"],
    }

    if len(slots) > 1:
        stats["slot 2"] = slots.get(2)

        # Move "slot 2" after "slot 1" in the dictionary
        new_stats = {}
        for key, value in stats.items():
            if key == "slot 1":
                new_stats[key] = value
                if "slot 2" in stats:
                    new_stats["slot 2"] = stats["slot 2"]
            elif key != "slot 2":
                new_stats[key] = value

        stats = new_stats

    pokemon_list.append(stats)





In [ ]:
import operator

sorted_pokemon_list = sorted(pokemon_list, key=operator.itemgetter("id"))
print(sorted_pokemon_list)

In [ ]:
import operator

#sorted_pokemon = sorted(pokemon_list, key=operator.itemgetter('id'))
sorted_pokemon = pokemon_list
                        
increment = 20
chunks = [sorted_pokemon[i:i+increment] for i in range(0, len(sorted_pokemon), increment)]

for i, chunk in enumerate(chunks):
    with open(f"pokemon_file_{i+1}.json", "w") as f:
        json.dump(chunk, f)

In [ ]:
import aiohttp
import asyncio
import time

pokemon_url = "https://pokeapi.co/api/v2/pokemon"
pokemon_list = []

tasks = []
not_found = []
async with aiohttp.ClientSession() as session:
    for pokemon in common_pokemon:
        while True:
            try:
                async with session.get(f"{pokemon_url}/{pokemon}") as response:
                    pokemon_data = await response.json()
                    break
            except aiohttp.ClientConnectionError:
                time.sleep(5)
                continue
            except aiohttp.ContentTypeError:
                not_found.append(pokemon)
                break

        wanted_slots = [1, 2]  # list
        available_slots = [type_dict["slot"] for type_dict in pokemon_data["types"]]

        # check =  all(item in available_slots  for item in wanted_slots)
        slots = {
            type_dict.get("slot"): type_dict.get("type").get("name")
            for type_dict in pokemon_data["types"]
            if type_dict.get("slot") in wanted_slots
        }
        pokemon_dict = {
            "name": pokemon_data["name"].capitalize(),
            "id": pokemon_data["id"],
            "base_experience": pokemon_data["base_experience"],
            "weight": pokemon_data["weight"],
            "height": pokemon_data["height"],
            "order": pokemon_data["order"],
            "slot 1": slots.get(1),
            "bmi": round(pokemon_data["weight"] / (pokemon_data["height"] ** 2), 4),
            "front_default": pokemon_data["sprites"]["front_default"],
        }

        # remove slot 2 is empty
        # only add slot 2 if it exists
        # print(len(slots))

        if len(slots) > 1:
            pokemon_dict["slot 2"] = slots.get(2)

        pokemon_list.append(pokemon_dict)

print(len(pokemon_list))


In [ ]:
import aiohttp
import asyncio
import time

pokemon_url = "https://pokeapi.co/api/v2/pokemon"
pokemon_list = []


async def fetch_pokemon_data(session, pokemon):
    async with session.get(f"{pokemon_url}/{pokemon}") as response:
        return await response.json()


async def get_pokemon_data(pokemon):
    async with aiohttp.ClientSession() as session:
        while True:
            try:
                pokemon_data = await fetch_pokemon_data(session, pokemon)
                break
            except aiohttp.ClientConnectionError:
                time.sleep(5)
                continue

        wanted_slots = [1, 2]  # list
        available_slots = [type_dict["slot"] for type_dict in pokemon_data["types"]]

        # check =  all(item in available_slots  for item in wanted_slots)
        slots = {
            type_dict.get("slot"): type_dict.get("type").get("name")
            for type_dict in pokemon_data["types"]
            if type_dict.get("slot") in wanted_slots
        }
        pokemon_dict = {
            "name": pokemon_data["name"].capitalize(),
            "id": pokemon_data["id"],
            "base_experience": pokemon_data["base_experience"],
            "weight": pokemon_data["weight"],
            "height": pokemon_data["height"],
            "order": pokemon_data["order"],
            "slot 1": slots.get(1),
            "bmi": round(pokemon_data["weight"] / (pokemon_data["height"] ** 2), 4),
            "front_default": pokemon_data["sprites"]["front_default"],
        }

        # remove slot 2 is empty
        # only add slot 2 if it exists
        # print(len(slots))

        if len(slots) > 1:
            pokemon_dict["slot 2"] = slots.get(2)

        return pokemon_dict


async def main():
    tasks = []
    async with aiohttp.ClientSession() as session:
        for pokemon in common_pokemon:
            tasks.append(get_pokemon_data(pokemon))

        pokemon_list.extend(await asyncio.gather(*tasks))

    print(len(pokemon_list))


if __name__ == "__main__":
    common_pokemon = ["pikachu", "charmander", "bulbasaur", "squirtle"]
    asyncio.run(main())


In [ ]:
pokemon_red = set(pokemon_by_version["red"])
pokemon_blue = set(pokemon_by_version["blue"])
pokemon_leaf_green = set(pokemon_by_version["leafgreen"])
pokemon_white = set(pokemon_by_version["white"])

print(len(pokemon_red))
print(len(pokemon_blue))
print(len(pokemon_leaf_green))
print(len(pokemon_white))

common_pokemon = pokemon_red | pokemon_blue | pokemon_leaf_green | pokemon_white


In [ ]:
# gdpr
import mimesis
from mimesis.locales import Locale

sample = {
    "name": "Pidgeot",
    "id": 18,
    "base_experience": 216,
    "weight": 395,
    "height": 15,
    "order": 23,
    "slot 1": "normal",
    "slot 2": "flying",
    "bmi": 1.7556,
    "front_default": "https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/18.png",
}

name = mimesis.Person(Locale.EN)
full_name = name.full_name()
print(full_name.)